## **Setting up Development Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [3]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 636 kB 48.5 MB/s 
     |████████████████████████████████| 3.3 MB 23.4 MB/s 
     |████████████████████████████████| 895 kB 32.6 MB/s 
     |████████████████████████████████| 1.2 MB 5.0 MB/s 


## **Importing Libraries**

In [29]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import RobertaTokenizer, RobertaModel

import pickle

## **Setting up Stemmer**

In [5]:
! rm -r tamil-stemmer-build
! rm input.txt
! rm output.txt

! mkdir tamil-stemmer-build
% cd tamil-stemmer-build
! git clone https://github.com/rdamodharan/tamil-stemmer.git
! tar -zxf tamil-stemmer/snowball-with-tamil.tgz
% cd snowball-with-tamil
! make

% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/My Drive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build
Cloning into 'tamil-stemmer'...
remote: Enumerating objects: 88, done.
remote: Total 88 (delta 0), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (88/88), done.
/content/drive/My Drive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build/snowball-with-tamil
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/space.o compiler/space.c
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/tokeniser.o compiler/tokeniser.c
In file included from compiler/tokeniser.c:6:0:
compiler/tokeniser.c: In function ‘read_token’:
compiler/header.h:8:21: warning: this statement may fall through [-Wimplicit-fallthrough=]
 #define unless(C) if(!(C))
                     ^
compiler/tokeniser.c:390:16: note: in expansion of macro ‘unless’
                unless (t->next == 0) {
             

## **Loading Data**

In [6]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [7]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [8]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam1490,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [9]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [10]:
dataframe.shape

(5880, 3)

In [11]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [12]:
dataframe.shape

(5877, 3)

In [13]:
text = dataframe['text']

In [14]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [15]:
label = dataframe['category']

In [16]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [17]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [18]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

In [19]:
text = text.tolist()

! rm input.txt
! rm output.txt

for i in text:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

rm: cannot remove 'input.txt': No such file or directory
rm: cannot remove 'output.txt': No such file or directory


In [20]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [21]:
text_file = open("output.txt", "r")

text = []

for line in text_file:
    text.append(line.strip())

In [22]:
# text = np.array(text)
# text

## **Feature Extraction**

In [30]:
# Load Transformer Model

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [32]:
print(tokenized_input)

{'input_ids': tensor([[    0, 12798,  2840, 10470, 12798,  2840,  9470, 12798,  2840,  7487,
         12798, 10965, 10172, 12798,  2840,  2840, 12798,  2840, 14292, 12798,
         10965, 23133,  1437, 12798,  2840, 11423, 12798,  2840,  4726, 12798,
          2840, 10965, 12798,  2840, 15722, 12798, 10965,  8384, 12798,  2840,
         15722, 12798,  2840,  7487, 12798, 10965,  8384,  1437, 12798,  2840,
         10278, 12798, 10965,  6382, 12798,  2840,  7487, 12798,  2840,  8906,
         12798, 10965, 23133,  1437, 12798,  2840, 15113, 12798,  2840,  4726,
         12798,  2840,  7487, 12798, 10965,  8384, 12798,  2840, 10278, 12798,
          2840,  4726, 12798,  2840, 15722,  1437, 12798,  2840, 10278, 12798,
          2840,  4333, 12798,  2840,  2840, 12798, 10965,  8384,  1437, 12798,
          2840,  8906, 12798, 10965, 27819, 12798,  2840, 15389, 12798, 10965,
          8384, 12798,  2840, 15389, 12798,  2840,  9470,  1437, 12798,  2840,
         10278, 12798, 10965, 27819, 1

In [33]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0039,  0.0732, -0.0400,  ..., -0.0309, -0.0469, -0.0173],
         [-0.1370, -0.4975, -0.1186,  ..., -0.1110,  0.0411,  0.1711],
         [-0.0135,  0.1127, -0.1689,  ..., -0.1682,  0.1643,  0.1893],
         ...,
         [ 0.1951, -0.2270, -0.0073,  ..., -0.0309, -0.2991,  0.4141],
         [ 0.1183, -0.1256,  0.1198,  ...,  0.4258,  0.1653,  0.2634],
         [ 0.0107,  0.0661, -0.0793,  ..., -0.0611, -0.0442, -0.0398]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 1.8189e-02, -1.9545e-01, -2.0222e-01, -6.7963e-02,  1.2369e-01,
          1.9648e-01,  2.6819e-01, -1.1820e-01, -1.3688e-01, -1.9866e-01,
          2.6931e-01, -4.0466e-03, -1.3773e-01,  1.2174e-01, -1.4257e-01,
          4.8432e-01,  1.9654e-01, -4.8852e-01, -3.5766e-04, -3.7526e-02,
         -2.5927e-01,  6.2809e-02,  4.6831e-01,  3.6818e-01,  1.2923e-01,
          4.0618e-02, -1.8181e-01, -1.2295e-02,  1.9235e-01,  2.6490

In [34]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [37]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [38]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./XLM-Roberta-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./XLM-Roberta-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./XLM-Roberta-Embedded-TrainData/output1.pickle done
./XLM-Roberta-Embedded-TrainData/output2.pickle done
./XLM-Roberta-Embedded-TrainData/output3.pickle done
./XLM-Roberta-Embedded-TrainData/output4.pickle done
./XLM-Roberta-Embedded-TrainData/output5.pickle done
./XLM-Roberta-Embedded-TrainData/output6.pickle done
./XLM-Roberta-Embedded-TrainData/output7.pickle done
./XLM-Roberta-Embedded-TrainData/output8.pickle done
./XLM-Roberta-Embedded-TrainData/output9.pickle done
./XLM-Roberta-Embedded-TrainData/output10.pickle done
./XLM-Roberta-Embedded-TrainData/output11.pickle done
./XLM-Roberta-Embedded-TrainData/output12.pickle done
./XLM-Roberta-Embedded-TrainData/output13.pickle done
./XLM-Roberta-Embedded-TrainData/output14.pickle done
./XLM-Roberta-Embedded-TrainData/output15.pickle done
./XLM-Roberta-Embedded-TrainData/output16.pickle done
./XLM-Roberta-Embedded-TrainData/output17.pickle done
./XLM-Roberta-Embedded-TrainData/output18.pickle done
./XLM-Roberta-Embedded-TrainData/outp

In [39]:
# Loading Embedded Input Data from Disk

output = []

for i in range(30):
    file_name = "./XLM-Roberta-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./XLM-Roberta-Embedded-TrainData/output1.pickle done
./XLM-Roberta-Embedded-TrainData/output2.pickle done
./XLM-Roberta-Embedded-TrainData/output3.pickle done
./XLM-Roberta-Embedded-TrainData/output4.pickle done
./XLM-Roberta-Embedded-TrainData/output5.pickle done
./XLM-Roberta-Embedded-TrainData/output6.pickle done
./XLM-Roberta-Embedded-TrainData/output7.pickle done
./XLM-Roberta-Embedded-TrainData/output8.pickle done
./XLM-Roberta-Embedded-TrainData/output9.pickle done
./XLM-Roberta-Embedded-TrainData/output10.pickle done
./XLM-Roberta-Embedded-TrainData/output11.pickle done
./XLM-Roberta-Embedded-TrainData/output12.pickle done
./XLM-Roberta-Embedded-TrainData/output13.pickle done
./XLM-Roberta-Embedded-TrainData/output14.pickle done
./XLM-Roberta-Embedded-TrainData/output15.pickle done
./XLM-Roberta-Embedded-TrainData/output16.pickle done
./XLM-Roberta-Embedded-TrainData/output17.pickle done
./XLM-Roberta-Embedded-TrainData/output18.pickle done
./XLM-Roberta-Embedded-TrainData/outp

In [40]:
X = np.array(X)
X.shape

(5877, 1, 768)

In [41]:
X = X.reshape(5877, 768)
X.shape

(5877, 768)

In [42]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [43]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
X_train.shape

(4701, 768)

In [45]:
X_val.shape

(1176, 768)

## **Logistic Regression**

In [46]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Support Vector Machine**

In [47]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [48]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.84      0.67      0.74       950
           1       0.25      0.47      0.33       226

    accuracy                           0.63      1176
   macro avg       0.55      0.57      0.54      1176
weighted avg       0.73      0.63      0.66      1176



## **Stochastic Gradient Descent**

In [49]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **K Nearest Neighbours**

In [50]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.82      0.94      0.87       950
           1       0.32      0.12      0.18       226

    accuracy                           0.78      1176
   macro avg       0.57      0.53      0.53      1176
weighted avg       0.72      0.78      0.74      1176



## **Decision tree**

In [51]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.84      0.82      0.83       950
           1       0.30      0.32      0.31       226

    accuracy                           0.73      1176
   macro avg       0.57      0.57      0.57      1176
weighted avg       0.73      0.73      0.73      1176



## **Random Forest**

In [52]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90       950
           1       0.75      0.04      0.08       226

    accuracy                           0.81      1176
   macro avg       0.78      0.52      0.49      1176
weighted avg       0.80      0.81      0.74      1176



## **Majority Voting**

In [54]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Using only KNN, DT & RF in predictions due to poor performance of other models
    predictions = [knn_pred[i], dt_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))    

              precision    recall  f1-score   support

           0       0.82      0.98      0.89       950
           1       0.45      0.08      0.13       226

    accuracy                           0.80      1176
   macro avg       0.63      0.53      0.51      1176
weighted avg       0.75      0.80      0.74      1176



## **Loading Testing Data**

In [55]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [56]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_565,any இரும்பு கோட்டை முரட்டு சிங்கம் & ஜெய்சங்கர...
freq,1,1


In [57]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [58]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

In [59]:
text_test = text_test.tolist()

! rm input.txt
! rm output.txt

for i in text_test:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

In [60]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [61]:
text_file = open("output.txt", "r")

text_test = []

for line in text_file:
    text_test.append(line.strip())

text_test = np.array(text_test)

## **Test Feature Extraction**

In [62]:
# Load Transformer Model

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [63]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [64]:
print(tokenized_input)

{'input_ids': tensor([[    0, 12798,  2840, 10470, 12798,  2840, 14292, 12798, 10965, 23133,
         12798,  2840,  8906, 12798,  2840,  4726,  7031,  1437, 12798,  2840,
          6382, 12798,  2840, 10470, 12798, 10965, 10172, 12798,  2840, 15722,
         12798, 10965,  8384, 12798,  2840, 15722, 12798, 10965, 10172, 12798,
          2840, 10470, 12798,  2840,  4726, 12798,  2840, 15375, 12798, 10965,
          8384,  1437, 12798,  2840, 15722, 12798,  2840, 10470, 12798, 10965,
         10172, 12798,  2840,  7487, 12798, 10965, 10172, 12798,  2840, 11423,
         12798, 10965,  8384, 12798,  2840, 10470, 12798, 10965, 13859, 12798,
          2840,  2840, 12798, 10965,  8384,  1437, 12798,  2840,  2840, 12798,
          2840,  4726, 12798,  2840, 18537, 12798, 10965,  8384,  1437, 12798,
          2840, 10470, 12798,  2840, 14292, 12798, 10965, 23133, 12798,  2840,
          8906, 12798,  2840,  4726,  3392,  1343,   740, 38635,  1437, 12798,
          2840,  4333, 12798, 10965,  

In [65]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0087,  0.0409, -0.0399,  ..., -0.0355, -0.0559, -0.0055],
         [-0.1651, -0.5570, -0.1394,  ..., -0.1705,  0.0282,  0.1733],
         [-0.0248,  0.1692, -0.2043,  ..., -0.1961,  0.1439,  0.2443],
         ...,
         [ 0.2110, -0.2285, -0.0590,  ..., -0.0766, -0.3051,  0.3497],
         [ 0.1942, -0.1283,  0.1386,  ...,  0.4634,  0.2046,  0.2245],
         [ 0.0075,  0.0335, -0.0794,  ..., -0.0743, -0.0499, -0.0268]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 1.8173e-02, -1.9578e-01, -1.9829e-01, -7.6081e-02,  1.1814e-01,
          1.9643e-01,  2.6295e-01, -1.3266e-01, -1.3908e-01, -1.9219e-01,
          2.5664e-01, -5.9377e-03, -1.3126e-01,  1.0252e-01, -1.2113e-01,
          4.8984e-01,  1.9830e-01, -4.8520e-01, -6.2342e-03, -3.1454e-02,
         -2.5866e-01,  6.9748e-02,  4.6906e-01,  3.6212e-01,  1.3399e-01,
          4.9350e-02, -1.7706e-01, -1.5896e-02,  1.7868e-01,  2.4945

In [66]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [68]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [69]:
# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./XLM-Roberta-Embedded-TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./XLM-Roberta-Embedded-TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./XLM-Roberta-Embedded-TestData/output1.pickle done
./XLM-Roberta-Embedded-TestData/output2.pickle done
./XLM-Roberta-Embedded-TestData/output3.pickle done
./XLM-Roberta-Embedded-TestData/output4.pickle done


In [70]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(4):
    file_name = "./XLM-Roberta-Embedded-TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

./XLM-Roberta-Embedded-TestData/output1.pickle done
./XLM-Roberta-Embedded-TestData/output2.pickle done
./XLM-Roberta-Embedded-TestData/output3.pickle done
./XLM-Roberta-Embedded-TestData/output4.pickle done


In [71]:
X_test = np.array(X_test)
X_test.shape

(654, 1, 768)

In [72]:
X_test = X_test.reshape(654, 768)
X_test.shape

(654, 768)

## **Getting Predictions**

In [73]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [74]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [knn_test_pred[i], dt_test_pred[i], rf_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [75]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [76]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [77]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [78]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.88      0.70      0.78       536
           1       0.30      0.58      0.39       118

    accuracy                           0.68       654
   macro avg       0.59      0.64      0.59       654
weighted avg       0.78      0.68      0.71       654



In [80]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [81]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.83      0.95      0.89       536
           1       0.38      0.13      0.19       118

    accuracy                           0.80       654
   macro avg       0.60      0.54      0.54       654
weighted avg       0.75      0.80      0.76       654



In [82]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       536
           1       0.31      0.34      0.32       118

    accuracy                           0.74       654
   macro avg       0.58      0.59      0.58       654
weighted avg       0.75      0.74      0.75       654



In [83]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.82      0.99      0.90       536
           1       0.40      0.03      0.06       118

    accuracy                           0.82       654
   macro avg       0.61      0.51      0.48       654
weighted avg       0.75      0.82      0.75       654



In [84]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.83      0.99      0.90       536
           1       0.65      0.11      0.19       118

    accuracy                           0.83       654
   macro avg       0.74      0.55      0.55       654
weighted avg       0.80      0.83      0.78       654



## **Saving the Predictions**

In [85]:
# Logistic Regression

predictions = le.inverse_transform(lr_test_pred)

dataframe_test_lr = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_lr = pd.DataFrame(dataframe_test_lr)
dataframe_test_lr.to_csv("Output/XLM-Roberta-LogisticRegression.csv")
dataframe_test_lr

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [86]:
# Support Vector Machine

predictions = le.inverse_transform(svm_test_pred)

dataframe_test_svm = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_svm = pd.DataFrame(dataframe_test_svm)
dataframe_test_svm.to_csv("Output/XLM-Roberta-SupportVectorMachine.csv")
dataframe_test_svm

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [87]:
# Naive Bayes

predictions = le.inverse_transform(nb_test_pred)

dataframe_test_nb = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_nb = pd.DataFrame(dataframe_test_nb)
dataframe_test_nb.to_csv("Output/XLM-Roberta-NaiveBayes.csv")
dataframe_test_nb

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,OFF
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",OFF
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [88]:
# Stochastic Gradient Descent

predictions = le.inverse_transform(sgd_test_pred)

dataframe_test_sgd = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_sgd = pd.DataFrame(dataframe_test_sgd)
dataframe_test_sgd.to_csv("Output/XLM-Roberta-StochasticGradientDescent.csv")
dataframe_test_sgd

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [89]:
# K Nearest Neighbours

predictions = le.inverse_transform(knn_test_pred)

dataframe_test_knn = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_knn = pd.DataFrame(dataframe_test_knn)
dataframe_test_knn.to_csv("Output/XLM-Roberta-KNearestNeighbours.csv")
dataframe_test_knn

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,OFF
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [90]:
# Decision Tree

predictions = le.inverse_transform(dt_test_pred)

dataframe_test_dt = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_dt = pd.DataFrame(dataframe_test_dt)
dataframe_test_dt.to_csv("Output/XLM-Roberta-DecisionTree.csv")
dataframe_test_dt

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",OFF
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [91]:
# Random Forest

predictions = le.inverse_transform(rf_test_pred)

dataframe_test_rf = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_rf = pd.DataFrame(dataframe_test_rf)
dataframe_test_rf.to_csv("Output/XLM-Roberta-RandomForest.csv")
dataframe_test_rf

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [92]:
# Majority Voting

predictions = le.inverse_transform(mv_test_pred)

dataframe_test_mv = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_mv = pd.DataFrame(dataframe_test_mv)
dataframe_test_mv.to_csv("Output/XLM-Roberta-MajorityVoting.csv")
dataframe_test_mv

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT
